<a href="https://www.kaggle.com/code/angevalli/weighted-max-sat-problems-solver?scriptVersionId=133930319" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a><a target="_blank" href="https://drive.google.com/drive/folders/1iEENoPi9mDsgYQRdjFj_OaQHW8ITR6hH?usp=sharing"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this notebook is to solve Weighted MAX SAT problems.


=== Purpose ===

Given a set of rules with weights, the goal of is to find the KB with the highest weight.
If there are several, find the one with the least number of atoms.

input a set of rules:
    likesHermioneRon => -likesHermioneHarry [100]
    => hermioneIsSmart [1]
    hermioneIsSmart & harryIsSmart => likesHermioneHarry [3]
    => harryIsSmart [10]
    => likesHermioneRon [20]

output the kb with the highest weight (in this case, the weight value is 133.0):
    harryIsSmart
    likesHermioneRon

Note: each line is a positive atom, we only output positive ones and assume the other atoms are negative.

In order to ensure a fair evaluation, do not use any non-standard Python libraries.

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/max-sat-problems-with-weights/weights.txt
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v20c40-2.txt
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v10c20-2.txt
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v100c400-10.txt
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v50c100-8.txt
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v100c400-8.txt
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/hermione.txt
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v20c40-4.txt
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v20c40-1.txt
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v100c1200_1.txt
/kaggle/input/max-sat-problems-with-weights/problems-20

In [2]:
from utility_script_weighted_max_sat_problems_solving import evaluate, evaluate_score, read_clause_from_file, timeout

In [3]:
import os
from itertools import combinations

# problem path, which includes a set of max sat problem text.
problem_path = "/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/"
# result path
result_path = "/kaggle/working/results/"
# weight path
weight_path = "/kaggle/input/max-sat-problems-with-weights/weights.txt"

In [4]:
def exhaustive_search_example():
    '''
    an example showing how to use the exhausitive search to solve the problem of 'hermione'
    :return:
    '''
    # read all clauses from this file
    clauses = read_clause_from_file(problem_path+'hermione.txt')

    # get all predicates
    predicates = set([atom.predicate for c in clauses for atom in c.atoms])

    # enumerate all possible sets of atoms
    sub_set = list()
    for i in range(0, len(predicates) + 1):
        temp = [list(x) for x in combinations(predicates, i)]
        sub_set.extend(temp)

    # use a loop to find the best kb
    best_weight, best_kb = 0.0, None
    for sub in sub_set:
        weight = 0.0
        for clause in clauses:
            if clause.is_satisfied_in(sub):
                weight += clause.weight
        if weight > best_weight:
            best_weight = weight
            best_kb = sub
    print('best kb = {a}'.format(a=best_kb))
    print('best weight = {b}'.format(b=best_weight))
    return best_kb

In [5]:
exhaustive_search_example()

best kb = ['likesHermioneRon', 'harryIsSmart']
best weight = 133.0


['likesHermioneRon', 'harryIsSmart']

In [6]:
@timeout(5 * 60)
def max_sat(clauses):
    '''
    Does not take longer than 5 minutes PER PROBLEM,
    otherwise this function will be stopped.
    :param clauses: a list of clauses
    :return:
    '''

    "unit propagation, random flipping, etc. -> SEE SLIDES TO KNOW WHAT TO IMPLEMENT"
    "Weighted CNF from Selman's wff generator"

    best_kb = set()
    treshold_nb_atoms = 25
    treshold_weights = 10

    # get dictionary of predicates
    dictionary_predicates = dict([(atom.predicate, atom) for c in clauses for atom in c.atoms])
    predicates = list(dictionary_predicates.keys())

    ######### Exhaustive search if there are few atoms
    if len(predicates) < treshold_nb_atoms :
      sub_set = list()
      for i in range(0, len(predicates) + 1):
          temp = [list(x) for x in combinations(predicates, i)]
          sub_set.extend(temp)

      best_weight, best_kb = 0.0, None
      for sub in sub_set:
          weight = 0.0
          for clause in clauses:
              if clause.is_satisfied_in(sub):
                  weight += clause.weight
          if weight > best_weight:
              best_weight = weight
              best_kb = sub

    ######### Unit propagation otherwise (with treshold on weights)
    else :
      best_kb = []
      for predicate in predicates :
        seen_predicate = False
        for clause in clauses :
          if seen_predicate : # In case the predicate has already been added to the database
              seen_atom = dictionary_predicates[predicate]
              if seen_atom.is_negative : # We remove the atom if it is negative
                clause = clause.atoms.remove(current_atom) if current_atom in clause.atoms else clause
              elif clause.is_satisfied_in(predicate) or clause.is_satisfied_in('-'+predicate) : # If the predicate or it's negation satisfies the clause, we remove it also 
                clause = clause.atoms.remove(current_atom) if current_atom in clause.atoms else clause
          if clause is not None : # clause can be None if we removed all atoms
            if clause.is_satisfied_in(predicate) and clause.weight > treshold_weights : # If the predicated is satisfied and the weight is high enough
                current_atom = dictionary_predicates[predicate]
                if current_atom.is_positive : # We keep only positive atoms
                  clause = clause.atoms.remove(current_atom) if current_atom in clause.atoms else clause #clause.atoms.remove(atom for atom in clause.atoms if atom.predicate == predicate) if predicate in clause.atoms.predicate else None
                  best_kb.append(predicate)
                else : # We remove negative ones
                  clause = clause.atoms.remove(current_atom) if current_atom in clause.atoms else clause
                seen_predicate = True

    return best_kb

def run():
    '''
    read problem file one by one, and apply your solution to it.
    :return:
    '''
    # create results folder
    if not os.path.exists(result_path):
        os.makedirs(result_path)

    for root, dirs, files in os.walk(problem_path):
        for name in files:
            # clause file
            file_name = os.path.join(root, name)
            print('file name = {a}'.format(a=file_name))
            # load all clauses in this file
            clauses = read_clause_from_file(file_name)
            # execute your solution
            best_kb = max_sat(clauses)
            # store the best kb
            with open(result_path+name.replace('.txt', '.res'), 'w', encoding='utf8')as f:
                for atom in best_kb:
                    f.write(atom + '\n')

In [7]:
run()

file name = /kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v20c40-2.txt
file name = /kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v10c20-2.txt
file name = /kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v100c400-10.txt
file name = /kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v50c100-8.txt
file name = /kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v100c400-8.txt
file name = /kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/hermione.txt
file name = /kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v20c40-4.txt
file name = /kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v20c40-1.txt
file name = /kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v100c1200_1.txt
fil

In [8]:
evaluate_score(result_path, problem_path, weight_path)

/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v20c40-2.txt:
  Custom score: 1.0085470085470085
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v10c20-2.txt:
  Custom score: 1.0227920227920229
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v100c400-10.txt:
  Custom score: 0.9512768146647929
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v50c100-8.txt:
  Custom score: 0.924450518800013
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v100c400-8.txt:
  Custom score: 0.9030451231030318
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/hermione.txt:
  Custom score: 0.9192042692525265
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v20c40-4.txt:
  Custom score: 0.9411799033400469
/kaggle/input/max-sat-problems-with-weights/problems-202306

In [9]:
evaluate(result_path, problem_path)

/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v20c40-2.txt:
  Weight: 236.0
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v10c20-2.txt:
  Weight: 112.0
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v100c400-10.txt:
  Weight: 1627.0
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v50c100-8.txt:
  Weight: 476.0
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v100c400-8.txt:
  Weight: 1764.0
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/hermione.txt:
  Weight: 133.0
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v20c40-4.txt:
  Weight: 191.0
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/problems/s2v20c40-1.txt:
  Weight: 245.0
/kaggle/input/max-sat-problems-with-weights/problems-20230617T162308Z-001/